In [3]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
# writer = SummaryWriter(log_dir="/Users/arponbiswas/Computer-Vision-Projects/Image_classification_projects/PC_Parts_Image_Classification/Tensorboard_Graph/Models_Graph/SimpleModel/v_1.0")
writer = True

In [5]:
train_trans = transforms.Compose([
    # 1. Geometric Transformations (with resizing)
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),  # Horizontal flips for left-right consistency
    transforms.RandomRotation(10),  # Minor rotation for realistic variation
    transforms.RandomAffine(
        degrees=0,  # No additional rotation
        translate=(0.05, 0.05),  # Small positional variance
        scale=(0.9, 1.1)  # Conservative scaling for proportion preservation
    ),
    transforms.RandomPerspective(distortion_scale=0.1, p=0.3, interpolation=3),  # Subtle 3D perspective

    # 2. Color Augmentations
    transforms.ColorJitter(
        brightness=0.15, contrast=0.15, saturation=0.15, hue=0.05
    ),  # Mild color variation for natural lighting

    # 3. Grayscale conversion, ToTensor, and Normalization for 1 channel
    transforms.Grayscale(num_output_channels=1),  # Convert to 1 channel
    transforms.ToTensor()
])

In [6]:
train_dataset = ImageFolder(root='/Users/arponbiswas/Computer-Vision-Projects/Image_classification_projects/PC_Parts_Image_Classification/Data/pc_parts_ready', transform=train_trans)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [7]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.version = '1.0'
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=6, stride=4, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3),
        )
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(1 * 21 * 21, 14)

    def forward(self, x):
        out = self.layer1(x)
        out = self.flatten(out)
        out = self.fc1(out)
        return out

In [8]:
model = SimpleCNN()
writer.add_graph(model, torch.rand(1, 1, 256, 256))
model

AttributeError: 'bool' object has no attribute 'add_graph'

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
def training_analysis(model, epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        all_preds = []
        all_labels = []
        
        progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}]", leave=False)
        
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            all_preds.extend(outputs.argmax(dim=1).tolist())
            all_labels.extend(labels.tolist())

            # Optional: show loss in the progress bar
            progress_bar.set_postfix(loss=loss.item())

        # Calculate metrics
        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
        precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
        recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
        cm = confusion_matrix(all_labels, all_preds)

        # Log to TensorBoard
        writer.add_scalar('Loss/train', epoch_loss / len(train_loader), epoch)
        writer.add_scalar('Accuracy/train', accuracy, epoch)
        writer.add_scalar('F1/train', f1, epoch)
        writer.add_scalar('Precision/train', precision, epoch)
        writer.add_scalar('Recall/train', recall, epoch)

        # Plot confusion matrix
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
        ax.set_xlabel('Predicted')
        ax.set_ylabel('True')
        ax.set_title('Confusion Matrix')
        writer.add_figure('Confusion Matrix', fig, global_step=epoch)
        plt.close(fig)

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss/len(train_loader):.4f} Accuracy: {accuracy:.4f} F1: {f1:.4f} Precision: {precision:.4f} Recall: {recall:.4f}")

In [11]:
# training_analysis(model, epochs=1)